In [135]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

In [115]:
test_df = pd.read_csv('test_target.csv')
rating_training_df = pd.read_csv('rating_encoded.csv')
movies_training_df = pd.read_csv('movies_encoded.csv')

In [116]:
test_df.head()

,userId,movieId,rating,datetime
0,514,162,4.0,2018-09-24 14:27:30
1,514,5246,1.5,2018-09-24 02:44:19
2,514,5247,2.5,2018-09-24 02:44:00
3,514,187595,3.0,2018-09-23 03:55:46
4,514,187031,2.5,2018-09-23 03:55:27


In [117]:
rating_training_df.head()

,userId,movieId,rating,datetime
0,475,3753,4.5,2017-06-21 07:17:30
1,475,2006,4.5,2017-06-21 07:17:25
2,475,33794,4.5,2017-06-21 07:17:01
3,475,59315,4.0,2017-06-21 07:16:55
4,475,122904,5.0,2017-06-21 07:16:51


In [118]:
movies_training_df.head()

,movieId,title,year,genres,(NO GENRES LISTED),ACTION,ADVENTURE,ANIMATION,CHILDREN,COMEDY,...,FILM-NOIR,HORROR,IMAX,MUSICAL,MYSTERY,ROMANCE,SCI-FI,THRILLER,WAR,WESTERN
0,1,TOY STORY (1995),1995,ADVENTURE|ANIMATION|CHILDREN|COMEDY|FANTASY,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,JUMANJI (1995),1995,ADVENTURE|CHILDREN|FANTASY,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,GRUMPIER OLD MEN (1995),1995,COMEDY|ROMANCE,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,4,WAITING TO EXHALE (1995),1995,COMEDY|DRAMA|ROMANCE,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,FATHER OF THE BRIDE PART II (1995),1995,COMEDY,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [119]:
training_df = pd.merge(rating_training_df, movies_training_df, how='left', on='movieId')

In [121]:
training_df.head()

,userId,movieId,rating,datetime,title,year,genres,(NO GENRES LISTED),ACTION,ADVENTURE,...,FILM-NOIR,HORROR,IMAX,MUSICAL,MYSTERY,ROMANCE,SCI-FI,THRILLER,WAR,WESTERN
0,475,3753,4.5,2017-06-21 07:17:30,"PATRIOT, THE (2000)",2000.0,ACTION|DRAMA|WAR,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,475,2006,4.5,2017-06-21 07:17:25,"MASK OF ZORRO, THE (1998)",1998.0,ACTION|COMEDY|ROMANCE,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,475,33794,4.5,2017-06-21 07:17:01,BATMAN BEGINS (2005),2005.0,ACTION|CRIME|IMAX,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,475,59315,4.0,2017-06-21 07:16:55,IRON MAN (2008),2008.0,ACTION|ADVENTURE|SCI-FI,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,475,122904,5.0,2017-06-21 07:16:51,DEADPOOL (2016),2016.0,ACTION|ADVENTURE|COMEDY|SCI-FI,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [122]:
training_df.isnull().sum()

userId                0
movieId               0
rating                0
datetime              0
title                 3
year                  3
genres                3
(NO GENRES LISTED)    3
ACTION                3
ADVENTURE             3
ANIMATION             3
CHILDREN              3
COMEDY                3
CRIME                 3
DOCUMENTARY           3
DRAMA                 3
FANTASY               3
FILM-NOIR             3
HORROR                3
IMAX                  3
MUSICAL               3
MYSTERY               3
ROMANCE               3
SCI-FI                3
THRILLER              3
WAR                   3
WESTERN               3
dtype: int64

In [123]:
training_df[training_df['title'].isnull()]

,userId,movieId,rating,datetime,title,year,genres,(NO GENRES LISTED),ACTION,ADVENTURE,...,FILM-NOIR,HORROR,IMAX,MUSICAL,MYSTERY,ROMANCE,SCI-FI,THRILLER,WAR,WESTERN
5186,462,162414,5.0,2016-11-07 03:32:11,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6234,448,143410,2.0,2016-09-18 18:53:22,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18939,528,40697,0.5,2014-02-07 01:27:47,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
training_df = training_df.dropna()

In [157]:
training_df.isnull().sum()

userId                0
movieId               0
rating                0
title                 0
year                  0
(NO GENRES LISTED)    0
ACTION                0
ADVENTURE             0
ANIMATION             0
CHILDREN              0
COMEDY                0
CRIME                 0
DOCUMENTARY           0
DRAMA                 0
FANTASY               0
FILM-NOIR             0
HORROR                0
IMAX                  0
MUSICAL               0
MYSTERY               0
ROMANCE               0
SCI-FI                0
THRILLER              0
WAR                   0
WESTERN               0
dt_year               0
dt_month              0
dt_day                0
dtype: int64

In [125]:
training_df.head()

,userId,movieId,rating,datetime,title,year,genres,(NO GENRES LISTED),ACTION,ADVENTURE,...,FILM-NOIR,HORROR,IMAX,MUSICAL,MYSTERY,ROMANCE,SCI-FI,THRILLER,WAR,WESTERN
0,475,3753,4.5,2017-06-21 07:17:30,"PATRIOT, THE (2000)",2000.0,ACTION|DRAMA|WAR,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,475,2006,4.5,2017-06-21 07:17:25,"MASK OF ZORRO, THE (1998)",1998.0,ACTION|COMEDY|ROMANCE,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,475,33794,4.5,2017-06-21 07:17:01,BATMAN BEGINS (2005),2005.0,ACTION|CRIME|IMAX,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,475,59315,4.0,2017-06-21 07:16:55,IRON MAN (2008),2008.0,ACTION|ADVENTURE|SCI-FI,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,475,122904,5.0,2017-06-21 07:16:51,DEADPOOL (2016),2016.0,ACTION|ADVENTURE|COMEDY|SCI-FI,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [126]:
training_df.datetime = pd.to_datetime(training_df.datetime)

In [127]:
training_df[['dt_year','dt_month','dt_day']] = [x.timetuple()[:3] for x in training_df.datetime.tolist()]

In [128]:
training_df = training_df.drop(['datetime', 'genres'], axis=1)

In [138]:
le = LabelEncoder()
training_df['title'] = le.fit_transform(training_df['title'])

In [139]:
training_df.head()

,userId,movieId,rating,title,year,(NO GENRES LISTED),ACTION,ADVENTURE,ANIMATION,CHILDREN,...,MUSICAL,MYSTERY,ROMANCE,SCI-FI,THRILLER,WAR,WESTERN,dt_year,dt_month,dt_day
0,475,3753,4.5,5756,2000.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2017,6,21
1,475,2006,4.5,4853,1998.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2017,6,21
2,475,33794,4.5,765,2005.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017,6,21
3,475,59315,4.0,3943,2008.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017,6,21
4,475,122904,5.0,2013,2016.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017,6,21


In [140]:
train_df, valid_df = train_test_split(training_df, random_state=0, train_size = 0.8, shuffle = True)

In [141]:
len(train_df), len(valid_df)

(72641, 18161)

In [142]:
y_train = train_df.rating
x_train = train_df.drop('rating', axis=1)
y_valid = valid_df.rating
x_valid = valid_df.drop('rating', axis = 1)

In [146]:
model = XGBRegressor(enable_categorical = True)
model.fit(x_train, y_train)
preds = model.predict(x_valid)
mse = mean_squared_error(preds, y_valid)

In [147]:
mse

np.float64(0.7421298582315152)

In [148]:
preds

array([2.1304471, 2.7315428, 3.5836654, ..., 4.3029375, 3.6834085,
       3.6911476], dtype=float32)

In [153]:
preds = np.round(preds*2)/2

In [155]:
mse = mean_squared_error(preds, y_valid)

In [156]:
mse

np.float64(0.7636556357028798)

In [158]:
test_df = pd.merge(test_df, movies_training_df, how='left', on='movieId')

In [159]:
test_df.head()

,userId,movieId,rating,datetime,title,year,genres,(NO GENRES LISTED),ACTION,ADVENTURE,...,FILM-NOIR,HORROR,IMAX,MUSICAL,MYSTERY,ROMANCE,SCI-FI,THRILLER,WAR,WESTERN
0,514,162,4.0,2018-09-24 14:27:30,CRUMB (1994),1994.0,DOCUMENTARY,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,514,5246,1.5,2018-09-24 02:44:19,SMOKEY AND THE BANDIT II (1980),1980.0,ACTION|COMEDY,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,514,5247,2.5,2018-09-24 02:44:00,SMOKEY AND THE BANDIT (1977),1977.0,ACTION|COMEDY,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,514,187595,3.0,2018-09-23 03:55:46,SOLO: A STAR WARS STORY (2018),2018.0,ACTION|ADVENTURE|CHILDREN|SCI-FI,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,514,187031,2.5,2018-09-23 03:55:27,JURASSIC WORLD: FALLEN KINGDOM (2018),2018.0,ACTION|ADVENTURE|DRAMA|SCI-FI|THRILLER,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [160]:
test_df.datetime = pd.to_datetime(test_df.datetime)
test_df[['dt_year','dt_month','dt_day']] = [x.timetuple()[:3] for x in test_df.datetime.tolist()]
test_df = test_df.drop(['datetime', 'genres'], axis=1)
le = LabelEncoder()
test_df['title'] = le.fit_transform(test_df['title'])

In [163]:
y_test = test_df.rating
x_test = test_df.drop('rating', axis = 1)

In [168]:
test_preds = model.predict(x_test)
test_preds = np.round(test_preds*2)/2
mse = mean_squared_error(test_preds, y_test)

In [165]:
mse

np.float64(1.542993156798572)